# SVM:- Support Vector Machine

In [1]:
import re
import nltk
import joblib
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report


In [2]:
bbc_data=pd.read_csv("C:\\Users\\ajlad\Downloads\\bbc_data.csv")

In [3]:
bbc_data['labels'].unique()

array(['entertainment', 'business', 'sport', 'politics', 'tech'],
      dtype=object)

In [4]:
bbc_data

,data,labels
0,Musicians to tackle US red tape Musicians gro...,entertainment
1,"U2s desire to be number one U2, who have won ...",entertainment
2,Rocker Doherty in on-stage fight Rock singer ...,entertainment
3,Snicket tops US box office chart The film ada...,entertainment
4,"Oceans Twelve raids box office Oceans Twelve,...",entertainment
...,...,...
2220,Warning over Windows Word files Writing a Mic...,tech
2221,Fast lifts rise into record books Two high-sp...,tech
2222,Nintendo adds media playing to DS Nintendo is...,tech
2223,Fast moving phone viruses appear Security fir...,tech


In [5]:
bbc_data.isnull().sum()

data      0
labels    0
dtype: int64

In [10]:
print(bbc_data.head())
print(bbc_data['data'].apply(type).unique())


                                                data         labels
0  [musicians, tackle, us, red, tape, musicians, ...  entertainment
1  [u2s, desire, number, one, u2, three, prestigi...  entertainment
2  [rocker, doherty, onstage, fight, rock, singer...  entertainment
3  [snicket, tops, us, box, office, chart, film, ...  entertainment
4  [oceans, twelve, raids, box, office, oceans, t...  entertainment
[<class 'list'>]


In [12]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold

# Load data
bbc_data = pd.read_csv("C:\\Users\\ajlad\\Downloads\\bbc_data.csv")


# Skip the preprocessing function as `bbc_data['data']` is already tokenized
def word_vectorizer(doc, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in doc:
        if word in model.wv:
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])
    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Train Word2Vec model
word2vec_model = Word2Vec(bbc_data['data'], vector_size=100, window=5, min_count=1, workers=4)

# Convert tokens to vectors
bbc_data['data'] = bbc_data['data'].apply(lambda x: word_vectorizer(x, word2vec_model, 100))

# Now you can use `bbc_data['data']` and `bbc_data['labels']` for further modeling or analysis


In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(bbc_data['data'],bbc_data['labels'],test_size=0.2,random_state=2)
# x_train=np.array(x_train)
# x_test=np.array(x_test)

In [27]:
from sklearn.preprocessing import StandardScaler

# Ensure x_train is a 2D array of shape (n_samples, n_features)
# Ensure y_train is a 1D array or list

# Optionally, standardize features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)

ValueError: setting an array element with a sequence.

In [26]:
from sklearn.svm import LinearSVC
sv_model=LinearSVC()
sv_model.fit(x_train_,y_train)

ValueError: setting an array element with a sequence.

In [ ]:
sv_model.predict(x_test)

array(['politics', 'tech', 'sport', 'business', 'business', 'business',
       'politics', 'sport', 'sport', 'entertainment', 'entertainment',
       'tech', 'tech', 'politics', 'business', 'entertainment',
       'politics', 'tech', 'sport', 'business', 'sport', 'tech', 'tech',
       'politics', 'entertainment', 'politics', 'business', 'sport',
       'tech', 'tech', 'sport', 'politics', 'politics', 'tech', 'tech',
       'sport', 'sport', 'politics', 'business', 'sport', 'sport',
       'sport', 'politics', 'sport', 'sport', 'tech', 'politics', 'sport',
       'entertainment', 'politics', 'tech', 'business', 'politics',
       'tech', 'sport', 'politics', 'business', 'tech', 'tech', 'tech',
       'business', 'entertainment', 'politics', 'tech', 'business',
       'sport', 'tech', 'business', 'politics', 'entertainment', 'tech',
       'sport', 'entertainment', 'entertainment', 'tech', 'entertainment',
       'business', 'tech', 'entertainment', 'tech', 'entertainment',
       'spor

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,sv_model.predict(x_test))

0.9842696629213483

In [ ]:
# Predict on training set
y_train_pred = sv_model.predict(x_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Predict on validation set
y_val_pred = sv_model.predict(x_test)
val_accuracy = accuracy_score(y_test, y_val_pred)

print(f'Training Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {val_accuracy}')

Training Accuracy: 1.0
Validation Accuracy: 0.9842696629213483


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
from sklearn.model_selection import cross_val_score
# Perform k-fold cross-validation
cross_val_scores = cross_val_score(sv_model, x, y, cv=kf)
print("Cross-validation scores:", cross_val_scores)
print("Mean CV accuracy:", cross_val_scores.mean())

Cross-validation scores: [0.98202247 0.97752809 0.97078652 0.97752809 0.98651685]
Mean CV accuracy: 0.978876404494382


In [ ]:
joblib.dump(sv_model,'sv_model.pkl')

['sv_model.pkl']

In [ ]:
bbc_data['labels'].value_counts()

labels
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64